<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment221114_recommedation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **CHAPTER 09 추천 시스템**
___


## **05 콘텐츠 기반 필터링 실습-TMDB 5000 영화 데이터 세트**
___
- **TMDB 5000 영화 데이터셋** : 유명한 영화 데이터 정보 사이트인 IMDB의 많은 영화 중 주요 5000개 영화에 대한 메타 정보를 새롭게 가공해 캐글에서 제공하는 데이터셋
- [TMDB 5000 영화 데이터셋 다운로드 링크](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata)


### | **장르 속성을 이용한 영화 콘텐츠 기반 필터링**
- 콘텐츠 기반 필터링은 **사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천**하는 것이다.
- 영화(혹은 상품이나 서비스) 간의 유사성을 판단하는 기준을 장르, 감독, 배우, 평점, 키워드 등 다양한 콘텐츠로 하는 방식이다.
- 콘텐츠 기반 필터링 추천 시스템을 영화 장르 속성을 기반으로 만들어보자. 장르 컬럼 값의 유사도를 비교한 뒤, 그중 높은 평점을 받은 영화를 추천한다.



### | **데이터 로딩 및 가공**
- 장르 속성을 이용하여 콘텐츠 기반 필터링을 수행해보자.
- 먼저 데이터셋을 DataFrame으로 로딩하고 간단하게 데이터를 살펴본다.

In [13]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [14]:
movies = pd.read_csv('/content/drive/MyDrive/2022-2 ESAA/Data/tmdb_5000_movies.csv')
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [15]:
print(movies.shape)

(4803, 20)


- `tmdb_5000_movies.csv`는 4803개의 관측치와 20개의 피처로 구성되어 있다.
- 영화 제목, 개요, 인기도, 평점, 투표 수, 예산, 키워드 등 영화에 대한 다양한 메타 정보를 가지고 있다.
- 이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 컬럼만 추출해 새롭게 DataFrame으로 만들어보자.
  -  `id`
  - `title` : 영화 제목
  - `genres` : 영화 장르
  - `vote_average` : 평균 평점
  - `vote_count` : 평점 투표 수
  - `popularity` : 영화의 인기도
  - `keywords` : 영화를 설명하는 주요 키워드 문구
  - `overview` : 영화에 대한 개요 설명

In [16]:
# 주요 컬럼 추출
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

- `genres`, `keywords` 등과 같은 컬럼을 보면 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열 값을 가지고 있다.
- 이 컬럼을 가공하여 필요한 정보를 추출하기 위하여, 먼저 해당 컬럼이 어떤 형태로 되어 있는지 확인해보자.

In [17]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- 위와 같이 `genres` 컬럼은 여러 개의 개별 장르 데이터를 가지고 있고, 이 개별 장르의 명칭은 딕셔너리의 키(key)인 'name'으로 추출할 수 있다.
- `keywords` 컬럼 역시 비슷한 구조를 가지고 있다.
- `genres` 컬럼의 문자열을 분해하여 개별 장르를 파이썬 리스트 객체로 추출하자. 파이썬 ast 모듈의 `literal_eval()` 함수를 이용하면 이 문자열을 문자열이 의미하는 `list[dict1, dict2]` 객체로 만들 수 있다.

In [18]:
from ast import literal_eval

# Series 객체의 apply()에 literal_eval 함수를 적용해 문자열을 객체로 반환
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

- 이제 `genres` 컬럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 가진다. 
- apply lambda 식을 이용하여 `genres` 컬럼에서 장르명만 리스트 객체로 추출해보자.

In [19]:
# 딕셔너리의 'name' 키에 해당하는 값을 찾아 리스트 객체로 변환
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x]) 
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

In [20]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### | **장르 콘텐츠 유사도 측정**
- `genres` 컬럼을 기반으로 하는 콘텐츠 기반 필터링을 다음과 같은 단계로 구현해보자.
  1. 문자열로 변환된 `genres` 컬럼을 Count 기반으로 피처 벡터화 변환한다.
  2. `genres` 문자열을 피처 벡터화 행렬로 변환한 데이터셋을 코사인 유사도를 이용하여 비교한다. 데이터셋의 관측치별로 다른 관측치와의 코사인 유사도 값을 가지는 객체를 생성한다.
  3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.
- 먼저 `genres` 컬럼을 문자열로 변환한 뒤 사이킷런의 CountVecorizer를 이용하여 피처 벡터 행렬로 만들어보자. 

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
# ('구분문자').join(리스트 객체) : 리스트 객체 내의 개별 값을 연속된 문자열로 반환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x)) 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


- CountVectorizer로 변환하여 4803개의 행과 276개의 개별 단어 피처로 구성된 피처 벡터 행렬이 만들어졌다.
- 이제 사이킷런의 `cosine_similarity()`를 이용하여 코사인 유사도를 계산하자.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape) # 코사인 유사도 행렬의 크기
print(genre_sim[:2]) # 코사인 유사도 행렬 데이터 2개 추출

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


- `cosine_similarities()` 호출로 생성된 `genre_sim` 객체는 `movies_df`의 `genre_literal` 컬럼을 피처 벡터화한 행렬 데이터의 행 별 유사도 정보를 가지고 있다. 즉, 데이터 프레임 `movies_df`의 행별 장르 유사도 값을 가지고 있는 것이다.
- `genre_sim` 객체에서 각 행별로 유사도 값이 높은 순으로 정렬하여, 위치 인덱스 값을 추출하자. 즉, 높은 유사도를 가진 행부터 인덱스를 추출한다.


In [24]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 유사도가 높은 순으로 행 인덱스 추출
print(genre_sim_sorted_ind[:1]) # 0번 관측치와 유사도가 높은 행 확인

[[   0 3494  813 ... 3038 3037 2401]]


- 0번 레코드의 경우, 자기 자신을 제외하면 3494번 레코드와 유사도가 가장 높고, 그다음이 813번 레코드이며, 가장 유사도가 낮은 레코드는 2401번 레코드이다.


### | **장르 콘텐츠 필터링을 위한 영화 추천**
- 이제 장르 유사도에 따라 영화를 추천하는 함수 `find_sim_movie()`를 생성해보자.
  + 데이터프레임 `movies_df`와 레코드별 코사인 유사도 인덱스를 가지고 있는 `genre_sim_sorted_ind`, 영화 제목, 추천할 영화 건수를 입력한다.
  + 추천 영화 정보를 가지는 데이터프레임을 만환한다.

In [25]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

- 위에서 만든 함수를 이용하여, 영화 '대부'와 장르별로 유사한 영화 10개를 추천해보자.

In [26]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10) # 영화 제목 입력
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- '대부 2편'이 1순위로 추천되었고, 대부와 비슷한 유형의 영화들이 추천되었다.
- 그러나 평점이 0인 'Mi America' 등 추천이 알맞지 않은 영화들이 몇 개 확인된다.
- 영화의 평점 정보인 `vote_average` 값을 이용하여, 좀 더 많은 추천 후보군을 선정한 후, 영화의 평점에 따라 최종 추천하는 방식으로 개선해보자.
- 그런데 이 컬럼은 소수의 관객이 특정 영화에 만점이나 매우 높은 평점을 부여하여 왜곡된 데이터를 가지고 있다. 이를 확인해보자.

In [27]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 'Stiff Upper Lips', 'Me You and Five Bucks' 등 영화 평점은 높으나 평가 횟수가 매우 작은 영화들이 있다.
- 이렇게 왜곡된 평점 데이터를 회피하기 위해 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다.
  + 가중치가 부여된 평점 방식(Weighted Rating)을 사용한다.
  + $Weighted Rating = (v/(v+m))*R + (m/(v+m))*C$
    - v : 개별 영화에 평점을 투표한 횟수
    - m : 평점을 부여하기 위한 최소 투표 횟수
    - R : 개별 영화에 대한 평균 평점
    - C : 전체 영화에 대한 평균 평점
  + m 값은 상위 60%에 해당하는 횟수를 기준으로 한다.


In [28]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 상위 60%
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


- 기존 평점을 새로운 가중 평점으로 변경하는 함수 `weighted_vote_average()`를 생성하고 이를 이용하여 새로운 평점 정보인 `vote_weighted` 값을 만들어보자.

In [29]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 

- 새롭게 부여된 `weighted_vote` 평점이 높은 순으로 상위 10개의 영화를 추출해보자.

In [30]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


- 이제 새롭게 정의된 평점 기준에 따라 영화를 추천해보자.
- 장르 유사성이 높은 영화를 2n개 뽑아 후보군으로 선정한 뒤, `weighted_vote` 컬럼 값이 높은 순으로 n개만큼 추출한다.

In [31]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [32]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
